In [1]:
################################################################
# テキストデータから、類似性検索のためのベクトルDBを作成する
################################################################

# 必要なライブラリをインストール
!pip install langchain tiktoken sentence_transformers faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.7 MB/s eta 0:00:00:00:0100:01


In [2]:
# テキストデータをチャンク化する
import tiktoken
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

filename = "data/nutanixbible.txt"

loader = TextLoader(filename, encoding='utf-8')
document = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=20,
)

splitted_texts = text_splitter.split_documents(document)
print(f"チャンクの総数:{len(splitted_texts)}")
print(f"チャンクされた文章の確認:\n{splitted_texts[6]}")


チャンクの総数:832
チャンクされた文章の確認:
page_content='私達は、単一のハイパーバイザー（ESXi）をサポートする単一のハードウェア プラットフォーム（NX）からスタートしましたが、常に単一のハイパーバイザー/プラットフォーム/クラウド企業以上の存在であることを認識していました。 これが、vCenter でプラグインではなく独自の UI をゼロから構築したり、カーネル内のネイティブなものではなくVM として実行したり、といった選択をした理由の1つです（他にも多くの理由があります）。 なぜでしょうか？ と聞かれるかもしれません。' metadata={'source': 'data/nutanixbible.txt'}


In [3]:

# Embeddingモデルを使ってベクトル化し、ベクトルDBを作成する
# https://huggingface.co/intfloat/multilingual-e5-large

import sentence_transformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 文章からベクトルに変換するためのモデル
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-Large')

# ベクトルデータベースを作成
db = FAISS.from_documents(splitted_texts, embeddings)

In [4]:
# ベクトルデータベースをファイルに出力
db.save_local("./data/vectorstore")


In [5]:
###################################################
# ベクトル検索

import time

question = "NutanixのAOSのアップグレードが失敗した場合の影響を教えてください。"

start = time.time()
# 質問に対して、データベース中の類似度上位3件を抽出。質問の文章はこの関数でベクトル化され利用される
docs = db.similarity_search(question, k=3)
elapsed_time = time.time() - start
print(f"処理時間[s]: {elapsed_time:.2f}")
for i in range(len(docs)):
    print(docs[i])

処理時間[s]: 0.03
page_content='ユーザー空間でVMとして実行する性質上、ハイパーバイザーの外部にあるため、アップグレードやCVMの「障害」といったものをエレガントに処理できます。 例えば、CVMがダウンするという破滅的な問題が発生した場合でも、ノード全体が、クラスタ内の他のCVMからのストレージI/Oとサービスで引き続き稼働します。 AOS（NutanixのCoreソフトウェア）のアップグレード中でも、そのホストで実行されているワークロードに影響を与えることなくCVMを再起動できます。' metadata={'source': 'data/nutanixbible.txt'}
page_content='Book of Prismの「Nutanixソフトウェアのアップグレード」および「ハイパーバイザーのアップグレード」セクションで、AOSおよびハイパーバイザーバージョンのアップグレードを実行するために使用される手順を説明しました。 このセクションでは、さまざまなタイプのアップグレードを無停止で実行できるようにする方法について説明します。\nAOSのアップグレードでは、いくつかの核となる手順が実行されます：' metadata={'source': 'data/nutanixbible.txt'}
page_content='例えば、停電またはノード障害が発生した場合、ローカルのNutanixクラスタでは、ストレージはローカル デバイスに永続化されており、ノードや電源がオンラインに戻ると復旧します。 AWSインスタンス ストアの場合、このケースには当てはまりません。\nほとんどの場合、AZ全体が電源を失うまたはダウンするという可能性は低いですが、 センシティブなワークロードの場合は、次のことをお勧めします：' metadata={'source': 'data/nutanixbible.txt'}
